In [2]:
from pathlib import Path
import numpy as np
from datetime import datetime, timedelta

from pyswb2.configuration import ModelConfig, GridConfig, InputConfig, OutputConfig
from pyswb2.model_domain import ModelDomain
from pyswb2.daily_calculation import DailyCalculation
from pyswb2.runoff import RunoffParameters
from pyswb2.soil import SoilParameters

def create_sample_data(nx: int, ny: int):
    """Create sample input grids"""
    # Create sample landuse grid (1=grass, 2=forest)
    landuse = np.ones((ny, nx), dtype=np.int32)
    landuse[:, nx//2:] = 2

    # Create sample soil groups (1=sandy, 2=clay)
    soils = np.ones((ny, nx), dtype=np.int32)
    soils[ny//2:, :] = 2

    # Create sample elevation grid (sloping from west to east)
    elevation = np.linspace(100, 200, nx)
    elevation = np.tile(elevation, (ny, 1))

    # Create canopy cover grid (50% for grass, 80% for forest)
    canopy_cover = np.where(landuse == 1, 0.5, 0.8)

    # Create D8 flow direction grid (flow to east)
    flow_dir = np.full((ny, nx), 1)

    return landuse, soils, elevation, canopy_cover, flow_dir

def create_sample_fragments_file():
    """Create a minimal fragments file for testing"""
    content = """# Sample fragments file
1 1 1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
2 1 1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
3 1 1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
4 1 1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
5 1 1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
6 1 1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
7 1 1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
8 1 1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
9 1 1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
10 1 1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
11 1 1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
12 1 1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0"""

    with open("fragments.txt", "w") as f:
        f.write(content)

    return Path("fragments.txt")


In [3]:
# Create sample fragments file
fragments_file = create_sample_fragments_file()

# Define model grid
grid_config = GridConfig(
    nx=10,
    ny=10,
    x0=0.0,
    y0=0.0,
    cell_size=1000.0  # 1km cells
)

# Create sample data
landuse, soils, elevation, canopy_cover, flow_dir = create_sample_data(grid_config.nx, grid_config.ny)

# Save sample grids
np.savetxt("landuse.asc", landuse)
np.savetxt("soils.asc", soils)
np.savetxt("elevation.asc", elevation)
np.savetxt("flow_dir.asc", flow_dir)

In [4]:
# Configure input/output
input_config = InputConfig(
    landuse_grid=Path("landuse.asc"),
    soils_grid=Path("soils.asc")
)

output_config = OutputConfig(
    directory=Path("output"),
    prefix="swb_sim",
    variables=["net_infiltration", "soil_storage", "actual_et"]
)

# Create model configuration
config = ModelConfig(
    grid=grid_config,
    input=input_config,
    output=output_config,
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2024, 12, 31)
)


In [5]:
# Initialize model
model = ModelDomain()

# Configure crop options
model.set_crop_options(
    dynamic_rooting=True,  # Enable if you want dynamic root depths
    use_crop_coefficients=True  # Enable if using FAO-56 crop coefficients
)

model.initialize_simulation_period(
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2024, 12, 31)
)

model.initialize_grid(
    nx=config.grid.nx,
    ny=config.grid.ny,
    x_ll=config.grid.x0,
    y_ll=config.grid.y0,
    cell_size=config.grid.cell_size
)

# Define parameters using dictionaries of raw values
# Define landuse parameters
landuse_params = {
    1: {  # Grass
        'runoff': {
            'curve_number': 70.0,
            'initial_abstraction_ratio': 0.2,
            'depression_storage': 0.1
        },
        'interception': {
            'canopy_storage_capacity': 0.05,
            'trunk_storage_capacity': 0.02,
            'stemflow_fraction': 0.1,
            'interception_storage_max_growing': 0.1,
            'interception_storage_max_nongrowing': 0.05
        }
    },
    2: {  # Forest
        'runoff': {
            'curve_number': 60.0,
            'initial_abstraction_ratio': 0.2,
            'depression_storage': 0.2
        },
        'interception': {
            'canopy_storage_capacity': 0.1,
            'trunk_storage_capacity': 0.05,
            'stemflow_fraction': 0.15,
            'interception_storage_max_growing': 0.2,
            'interception_storage_max_nongrowing': 0.1
        }
    }
}

soil_params = {
    1: {  # Sandy
        'infiltration': {
            'maximum_rate': 1.0,
            'minimum_rate': 0.1,
            'soil_storage_max': 8.0
        }
    },
    2: {  # Clay
        'infiltration': {
            'maximum_rate': 0.5,
            'minimum_rate': 0.05,
            'soil_storage_max': 12.0
        }
    }
}


soil_params = {
    1: {  # Sandy
        'infiltration': {
            'maximum_rate': 1.0,
            'minimum_rate': 0.1,
            'soil_storage_max': 8.0
        },
        'soil': {
            'field_capacity': 0.2,
            'wilting_point': 0.05,
            'hydraulic_conductivity': 1.0
        }
    },
    2: {  # Clay
        'infiltration': {
            'maximum_rate': 0.5,
            'minimum_rate': 0.05,
            'soil_storage_max': 12.0
        },
        'soil': {
            'field_capacity': 0.4,
            'wilting_point': 0.15,
            'hydraulic_conductivity': 0.5
        }
    }
}

# Initialize parameters
model.initialize_parameters(landuse_params, soil_params)


In [6]:
# Initialize modules
landuse_indices = landuse.flatten()
soil_indices = soils.flatten()

# Initialize interception module specifically
model.interception_module.initialize(
    landuse_indices=landuse_indices,
    canopy_cover=canopy_cover.flatten()
)

# Initialize agriculture module
model.agriculture_module.initialize(landuse_indices)

# Initialize other modules
model.initialize_modules(
    landuse_indices=landuse_indices,
    soil_indices=soil_indices,
    elevation=elevation.flatten(),
    latitude=np.full_like(elevation.flatten(), 45.0),
    fragments_file=fragments_file,
)


# Run daily calculations with proper date tracking
current_date = model.start_date
daily_calc = DailyCalculation(model)

# Set example precipitation
model.gross_precipitation = np.full(model.domain_size, 0.1)

# Update current date and run calculations

#daily_calc.perform_daily_calculation(current_date)


# Run simulation
#current_date = datetime(2024, 1, 1)
#end_date = datetime(2024, 12, 31)

In [7]:

while current_date <= config.end_date:
    # Get weather data (in practice this would come from files)
    model.tmin = np.full(model.domain_size, 10.0)  # Example temperature data
    model.tmax = np.full(model.domain_size, 20.0)
    model.gross_precipitation = np.full(model.domain_size, 0.1)  # Example precipitation

    # Run daily calculations
    model.update_date(current_date)
    daily_calc.perform_daily_calculation(current_date)

    # Write output (simplified)
    print(f"Date: {current_date}")
    print(f"Mean net infiltration: {model.net_infiltration.mean():.3f}")
    print(f"Mean soil storage: {model.soil_storage.mean():.3f}")
    print(f"Mean actual ET: {model.actual_et.mean():.3f}")
    print()

    current_date += timedelta(days=1)

Date: 2024-01-01 00:00:00
Mean net infiltration: 0.000
Mean soil storage: 0.000
Mean actual ET: 0.000

Date: 2024-01-02 00:00:00
Mean net infiltration: 0.000
Mean soil storage: 0.000
Mean actual ET: 0.000

Date: 2024-01-03 00:00:00
Mean net infiltration: 0.000
Mean soil storage: 0.000
Mean actual ET: 0.000

Date: 2024-01-04 00:00:00
Mean net infiltration: 0.000
Mean soil storage: 0.000
Mean actual ET: 0.000

Date: 2024-01-05 00:00:00
Mean net infiltration: 0.000
Mean soil storage: 0.000
Mean actual ET: 0.000

Date: 2024-01-06 00:00:00
Mean net infiltration: 0.000
Mean soil storage: 0.000
Mean actual ET: 0.000

Date: 2024-01-07 00:00:00
Mean net infiltration: 0.000
Mean soil storage: 0.000
Mean actual ET: 0.000

Date: 2024-01-08 00:00:00
Mean net infiltration: 0.000
Mean soil storage: 0.000
Mean actual ET: 0.000

Date: 2024-01-09 00:00:00
Mean net infiltration: 0.000
Mean soil storage: 0.000
Mean actual ET: 0.000

Date: 2024-01-10 00:00:00
Mean net infiltration: 0.000
Mean soil storage:

C:\Users\paulinkenbrandt\Documents\GitHub\pySWB2\pyswb2\interception.py:216: RuntimeWarning: divide by zero encountered in log
  np.log(1.0 - e_div_p[mask])
C:\Users\paulinkenbrandt\Documents\GitHub\pySWB2\pyswb2\daily_calculation.py:151: RuntimeWarning: invalid value encountered in divide
  (interim_moisture - self.domain.wilting_point) /
